In [48]:
import Pkg
Pkg.add("CSV")
Pkg.add("Plots")
Pkg.add("DataFrames")
Pkg.add("Statistics")
Pkg.add("LowRankModels")

  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`
  Resolving package versions...
No Changes to `~/.julia/environments/v1.5/Project.toml`
No Changes to `~/.julia/environments/v1.5/Manifest.toml`


In [49]:
using CSV
using Plots
using DataFrames
using Statistics
using LinearAlgebra
include("proxgrad.jl")

proxgrad_const

In [98]:
df = CSV.read("Police_Use_of_Force_onehot.csv")

,Column1,X,Y,PoliceUseOfForceID,CaseNumber,ResponseDate
,Int64,Float64,Float64,Int64,String,String
1,1,-93.2864,45.0101,15631493,08-018973,2008/01/19 22:53:35+00
2,2,-93.2661,44.9743,15631494,08-019237,2008/01/20 03:47:57+00
3,3,-93.272,44.9817,15631495,08-031728,2008/02/02 01:33:16+00
4,4,-93.269,44.9821,15631496,08-032796,2008/02/03 00:50:46+00
5,5,-93.2487,44.9529,15631497,08-043457,2008/02/14 19:53:39+00
6,6,-93.2989,44.9999,15631498,08-045814,2008/02/17 01:47:17+00
7,7,-93.2725,44.9796,15631499,08-053384,2008/02/25 02:22:01+00
8,8,-93.2779,44.9368,15631500,08-056530,2008/02/28 05:32:29+00
9,9,-93.2716,44.9766,15631501,08-061833,2008/03/04 13:32:34+00


In [99]:
feature_names = names(df)
for i in 1:95
    println(string(i), "\t", string(feature_names[i]), "\t\t\t", string(eltype(df[!, i])))
end

train_proportion = 0.8
n = size(df, 1)
println("Size of dataset: ", string(n))

# Put the first ntrain observations in the DataFrame df into the training set, and the rest into the test set
ntrain = convert(Int, round(train_proportion*n))
println(ntrain)

target = df[:, :ForceType]
data = df[:, filter(col -> (col != "ForceType" && col != "X" && col != "Y" && col != "PoliceUseOfForceID" 
            && col != "CaseNumber" && col != "ResponseDate"  && col != "DateAdded"  && col != "OBJECTID"
            && col != "ForceTypeAction_m_Body Weight to Pin" && col != "ForceTypeAction_m_Crowd Control Mace"
            && col != "ForceTypeAction_m_Firing Darts" && col != "ForceTypeAction_m_Joint Lock"
            && col != "ForceTypeAction_m_Kicks" && col != "ForceTypeAction_m_Knees"
            && col != "ForceTypeAction_m_Other" && col != "ForceTypeAction_m_Personal Mace" 
            && col != "ForceTypeAction_m_Punches" && col != "ForceTypeAction_m_Push Away" 
            && col != "ForceTypeAction_m_Takedown" && col != "Column1"), feature_names)]
#Now you will split the data to create training and test sets. 

# the following variable records the features of examples in the training set
train_x = data[1:ntrain,:]
# the following variable records the features of examples in the test set
test_x = data[ntrain+1:n,:]

# use to turn the force types into numbers (i.e. their index)
force_types = ["Bodily Force", "Chemical Irritant", "Taser", "Police K9 Bite", "Firearm", "Baton", "Improvised Weapon", 
    "Gun Point Display", "Maximal Restraint Technique", "Less Lethal", "Less Lethal Projectile"]
# the following variable records the labels of examples in the training set
train_y = []
for i = 1:ntrain
    append!(train_y, findall(x->x==target[i,1], force_types))
end
# the following variable records the labels of examples in the test set
test_y = []
for i = ntrain+1:n
    append!(test_y, findall(x->x==target[i,1], force_types))
end

# let's take a look
train_x

1	Column1			Int64
2	X			Float64
3	Y			Float64
4	PoliceUseOfForceID			Int64
5	CaseNumber			String
6	ResponseDate			String
7	ForceReportNumber			Int64
8	SubjectRoleNumber			Int64
9	ForceType			String
10	EventAge			String
11	TotalCityCallsForYear			Int64
12	TotalPrecinctCallsForYear			Int64
13	TotalNeighborhoodCallsForYear			Int64
14	CenterGBSID			Int64
15	CenterLatitude			Float64
16	CenterLongitude			Float64
17	CenterX			Float64
18	CenterY			Float64
19	DateAdded			String
20	OBJECTID			Int64
21	ResponseDate_year			Int64
22	ResponseDate_month			Int64
23	ResponseDate_day			Int64
24	ResponseDate_hour			Int64
25	ResponseDate_minute			Int64
26	Problem_m_Assault in Progress			Int64
27	Problem_m_Disturbance			Int64
28	Problem_m_Domestic Abuse-In Progress			Int64
29	Problem_m_Emotionally Disturb Person			Int64
30	Problem_m_Fight			Int64
31	Problem_m_Other			Int64
32	Problem_m_Suspicious Person			Int64
33	Problem_m_Suspicious Vehicle			Int64
34	Problem_m_Traffic Law Enforcement			Int64
35	Problem_

,ForceReportNumber,SubjectRoleNumber,EventAge,TotalCityCallsForYear,TotalPrecinctCallsForYear
,Int64,Int64,String,Int64,Int64
1,1,2,16,322402,80434
2,1,1,40,322402,46998
3,1,1,23,322402,46998
4,1,1,23,322402,46998
5,1,1,34,322402,84018
6,1,1,23,322402,80434
7,6,1,28,322402,46998
8,1,1,50,322402,55689
9,1,1,42,322402,46998


In [100]:
function string_to_float(str)
    try
        parse(Float64, str)
    catch
       0.0
    end
end

labels_string = [
    :EventAge
]

d = size(train_x,2)
d2 = size(test_x,2)

train_vals_from_string = convert(Matrix, string_to_float.(train_x[labels_string]))
train_x_no_string = hcat(convert(Matrix, train_x[:,1:2]), train_vals_from_string, convert(Matrix, train_x[:,4:d]))
X_train = hcat(train_x_no_string, ones(size(train_x,1)))

test_vals_from_string = convert(Matrix, string_to_float.(test_x[labels_string]))
test_x_no_string = hcat(convert(Matrix, test_x[:,1:2]), test_vals_from_string, convert(Matrix, test_x[:,4:d2]))
X_test = hcat(test_x_no_string, ones(size(test_x,1)))

5850×76 Array{Float64,2}:
  1.0   1.0  22.0  316150.0  51015.0  …  1.0  0.0  0.0  0.0  0.0  0.0  1.0
  1.0   1.0  30.0  316150.0  77032.0     0.0  0.0  1.0  0.0  0.0  0.0  1.0
  1.0   1.0  44.0  316150.0  79409.0     0.0  0.0  0.0  0.0  1.0  0.0  1.0
  3.0   1.0  16.0  316150.0  77032.0     0.0  0.0  1.0  0.0  0.0  0.0  1.0
  3.0   1.0  29.0  380794.0  86438.0     0.0  0.0  0.0  0.0  1.0  0.0  1.0
  1.0   1.0  32.0  380794.0  90815.0  …  0.0  0.0  0.0  0.0  1.0  0.0  1.0
  1.0   1.0  28.0  380794.0  73013.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
  1.0   1.0   0.0  380794.0  73013.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0
  1.0   1.0  63.0  380794.0  86438.0     0.0  0.0  0.0  0.0  1.0  0.0  1.0
  1.0   1.0  23.0  380794.0  90815.0     0.0  0.0  0.0  1.0  0.0  0.0  1.0
  1.0   1.0  31.0  380794.0  86438.0  …  0.0  0.0  0.0  0.0  1.0  0.0  1.0
  1.0   1.0  21.0  380794.0  56412.0     0.0  0.0  0.0  0.0  1.0  0.0  1.0
  1.0   1.0  50.0  380794.0  73013.0     1.0  0.0  0.0  0.0  0.0  0.0  1.0

In [101]:
m = length(train_y)
loss = OvALoss(11, bin_loss = LogisticLoss())
mul!(loss, 1/m)
w_ova = proxgrad(loss, ZeroReg(), X_train, train_y, stepsize=1/norm(X_train)^2, maxiters=200)

76×11 Array{Float64,2}:
  1.91869e-17  -3.20206e-17  -3.88734e-17  …  -4.67188e-17  -4.6889e-17
  1.36789e-17  -2.38267e-17  -3.02668e-17     -3.65747e-17  -3.67422e-17
  4.1998e-16   -6.72377e-16  -6.85704e-16     -8.59282e-16  -8.58766e-16
  4.42004e-12  -7.2462e-12   -8.17603e-12     -1.0069e-11   -1.00677e-11
  9.32808e-13  -1.51185e-12  -1.66674e-12     -2.04537e-12  -2.04506e-12
  1.29025e-13  -2.24787e-13  -3.32087e-13  …  -4.02431e-13  -4.02306e-13
  2.37733e-13  -3.89491e-13  -4.41241e-13     -5.44518e-13  -5.45055e-13
  6.057e-16    -9.93197e-16  -1.11438e-15     -1.37423e-15  -1.37494e-15
 -1.25603e-15   2.05962e-15   2.31089e-15      2.84979e-15   2.85126e-15
 -1.3982e-10    2.29276e-10   2.57247e-10      3.17238e-10   3.174e-10
  7.56572e-11  -1.24058e-10  -1.39195e-10  …  -1.71652e-10  -1.7174e-10
  2.71031e-14  -4.44415e-14  -4.98698e-14     -6.14945e-14  -6.15264e-14
  8.5991e-17   -1.41353e-16  -1.59549e-16     -1.96671e-16  -1.96692e-16
  ⋮                            

In [102]:
function MSE(y, pred)
    sum((y .- pred).*(y .- pred))/length(y)
end

train_pred_ova = X_train * w_ova
test_pred_ova = X_test * w_ova

train_MSE_ova = MSE(train_y, train_pred_ova)
test_MSE_ova = MSE(test_y, test_pred_ova)

println("train MSE: ", train_MSE_ova)
println("test MSE: ", test_MSE_ova)

train MSE: 48.55451668707802
test MSE: 55.0579553276346


In [103]:
loss = MultinomialLoss(11)
mul!(loss, 1/m)
w_mnl = proxgrad(loss, ZeroReg(), X_train, train_y, stepsize=1, maxiters=200)

76×11 Array{Float64,2}:
 -9.62067e-14   2.07402e-13  -5.91539e-14  …  -8.44324e-15  -2.38463e-14
 -2.05697e-13   2.8369e-13   -2.8543e-14      -4.46747e-15  -1.98384e-14
  4.85928e-12  -4.49947e-12   1.44167e-12     -5.15412e-13  -4.47008e-13
  8.41474e-9    8.5823e-9     3.5858e-9       -5.67497e-9   -5.34052e-9
  5.07122e-9   -2.09335e-9    1.73175e-10     -1.13849e-9   -1.06392e-9
 -4.25072e-9    6.5823e-9    -3.86006e-10  …  -2.47552e-10  -2.26761e-10
  3.10445e-10   7.27245e-10   3.19595e-10     -2.60408e-10  -3.0044e-10
  1.35185e-12   8.20115e-13   6.96905e-13     -6.98285e-13  -7.36454e-13
 -2.80023e-12  -1.70131e-12  -1.4484e-12       1.44789e-12   1.52724e-12
 -3.11721e-7   -1.89389e-7   -1.61235e-7       1.61179e-7    1.70011e-7
  1.68922e-7    1.02419e-7    8.69939e-8   …  -8.72256e-8   -9.19875e-8
  6.03878e-11   3.69478e-11   3.09022e-11     -3.12065e-11  -3.2953e-11
  1.29847e-13   1.87902e-13   8.72293e-14     -1.07267e-13  -1.0502e-13
  ⋮                               

In [104]:
train_pred_mnl = X_train * w_mnl
test_pred_mnl = X_test * w_mnl

train_MSE_mnl = MSE(train_y, train_pred_mnl)
test_MSE_mnl = MSE(test_y, test_pred_mnl)

println("train MSE: ", train_MSE_mnl)
println("test MSE: ", test_MSE_mnl)

train MSE: 94.95597017615492
test MSE: 101.45249199400112


In [147]:
# cross validation
validation_proportion = 0.2
n = size(X_train, 1)
println("Size of dataset: ", string(n))

# Put the first ntrain observations in the DataFrame df into the training set, and the rest into the test set
seg_train = convert(Int, round(validation_proportion*n))
println("size of seg train: ", seg_train)
segments = [];
y_segments = [];
for k = 1:4
#     println(1+(k-1)*(seg_train))
#     println((seg_train)*k)
    push!(segments, X_train[1+(k-1)*(seg_train):(seg_train)*k,:])
    push!(y_segments, train_y[1+(k-1)*(seg_train):(seg_train)*k,:])
end

push!(segments, X_train[1+(4)*(seg_train):n,:]);
push!(y_segments, train_y[1+(4)*(seg_train):n,:]);

for iter = 1:5
    validation_x = segments[iter]
    training_x = zeros(1,76)
    training_y = []
    println(size(y_segments))
    for inner = 1:5
        if inner != iter
            training_x = vcat(training_x, segments[inner])
            append!(training_y, y_segments[inner])
        end
    end
#     println(size(training_x))
    training_x = training_x[2:size(training_x,1),:]
#     println("size of training y", size(training_y))
    m = length(training_y)
    loss = OvALoss(11, bin_loss = LogisticLoss())
    mul!(loss, 1/m)
#     println(size(X_train))
#     println(size(training_x))
#     println(training_y)
    loss = MultinomialLoss(11)
    mul!(loss, 1/m)
    w_mnl = proxgrad(loss, ZeroReg(), training_x, training_y, stepsize=1, maxiters=200)
    
    train_pred_mnl = X_train * w_mnl
    test_pred_mnl = X_test * w_mnl

    train_MSE_mnl = MSE(train_y, train_pred_mnl)
    test_MSE_mnl = MSE(test_y, test_pred_mnl)

    println("train MSE: ", train_MSE_mnl)
    println("test MSE: ", test_MSE_mnl)
    
    m = length(train_y)
    loss = OvALoss(11, bin_loss = LogisticLoss())
    mul!(loss, 1/m)
    w_ova = proxgrad(loss, ZeroReg(), training_x, training_y, stepsize=1/norm(X_train)^2, maxiters=200)
    
    train_pred_ova = X_train * w_ova
    test_pred_ova = X_test * w_ova

    train_MSE_ova = MSE(train_y, train_pred_ova)
    test_MSE_ova = MSE(test_y, test_pred_ova)

    println("train MSE: ", train_MSE_ova)
    println("test MSE: ", test_MSE_ova)
    
end

Size of dataset: 23401
size of seg train: 4680
(5,)
train MSE: 95.5741462923422
test MSE: 102.07055773542855
train MSE: 48.52996325149712
test MSE: 55.032820906823396
(5,)
train MSE: 94.0411744072877
test MSE: 100.53776323874084
train MSE: 48.5299648758062
test MSE: 55.03282254800443
(5,)
train MSE: 93.99581639129703
test MSE: 100.49247414879079
train MSE: 48.529963628327266
test MSE: 55.03282133863616
(5,)
train MSE: 93.93045396020467
test MSE: 100.4271338249867
train MSE: 48.529962641537935
test MSE: 55.032820366641296
(5,)
train MSE: 101.15943645838534
test MSE: 107.65538468102892
train MSE: 48.5299592629519
test MSE: 55.03281681814603
